### Inference from a list of echo studies ###

In [1]:
import os
import glob
import pickle
import pandas as pd

# Custom imports
from werdich_cfr.tfutils.Modeltrainer import VideoTrainer

TensorFlow Version: 2.1.0


In [2]:
def read_model_dict(file):
    with open(file, 'rb') as f:
        model_dict = pickle.load(f)
    return model_dict

#%% Directories and data sets
cfr_data_root = os.path.normpath('/mnt/obi0/andreas/data/cfr')
log_dir = os.path.join(cfr_data_root, 'log', '20020a4c_pad')
test_data_dir = os.path.join(cfr_data_root, 'tfr_200202')
model_dict_file = glob.glob(os.path.join(log_dir, '*_model_dict.pkl'))[0]
model_dict = read_model_dict(model_dict_file)

# Re-create the model from checkpoint
checkpoint_file_list = sorted(glob.glob(os.path.join(log_dir, model_dict['name']+'*_chkpt_*')))
checkpoint_file = checkpoint_file_list[14]

In [3]:
# Re-create the model from checkpoint
model = load_model(checkpoint_file)
model.load_weights(checkpoint_file)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
video (InputLayer)              [(None, 30, 299, 299 0                                            
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, 28, 297, 297, 896         video[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 28, 297, 297, 128         conv3d[0][0]                     
__________________________________________________________________________________________________
max_pooling3d (MaxPooling3D)    (None, 28, 148, 148, 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [4]:
estimator = VideoTrainer(log_dir = log_dir,
                         model_dict = model_dict,
                         train_dict = None)

test_tfr_files = sorted(glob.glob(os.path.join(test_data_dir, 'CFR_200202_view_a4c_test_*.tfrecords')))
test_parquet_files = [file.replace('.tfrecords', '.parquet') for file in test_tfr_files]

batch_size = 10
n_take = 5

In [5]:
#parquet labels
test_df = pd.concat([pd.read_parquet(file) for file in test_parquet_files])
df_cfr_list = list(test_df.cfr.values[:n_take*batch_size])

#tfr files
n_steps_test, test_dataset = estimator.build_dataset(test_tfr_files,
                                                     batch_size=10,
                                                     buffer_n_batches=None,
                                                     repeat_count=1,
                                                     shuffle=False)
dset_cfr_list = []
for b, batch_output in enumerate(test_dataset.take(n_take)):
    print('Pulling batch {} of {}.'.format(b+1, n_take))
    image, labels = batch_output
    dset_cfr_list.extend(list(labels['score_output'].numpy()))

df_cfr = pd.DataFrame({'df': df_cfr_list,
                       'dset': dset_cfr_list})

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Pulling batch 1 of 5.
Pulling batch 2 of 5.
Pulling batch 3 of 5.
Pulling batch 4 of 5.
Pulling batch 5 of 5.


In [6]:
print(df_cfr)

          df      dset
0   2.404288  2.404288
1   2.488095  2.488095
2   1.599432  1.599432
3   2.066356  2.066356
4   0.870137  0.870137
5   1.642648  1.642648
6   1.424691  1.424691
7   1.563758  1.563758
8   2.404288  2.404288
9   2.144766  2.144766
10  1.259314  1.259314
11  2.038288  2.038288
12  1.647627  1.647627
13  2.075501  2.075501
14  2.240754  2.240754
15  2.607143  2.607143
16  1.309837  1.309837
17  1.194884  1.194884
18  1.661782  1.661783
19  1.142522  1.142522
20  0.679818  0.679818
21  2.000812  2.000812
22  0.933810  0.933810
23  1.225256  1.225256
24  1.939511  1.939511
25  0.834783  0.834783
26  1.418968  1.418968
27  2.496112  2.496112
28  2.385086  2.385086
29  1.432231  1.432230
30  1.259202  1.259202
31  1.606000  1.606000
32  1.412959  1.412959
33  1.030008  1.030007
34  2.038288  2.038288
35  1.615126  1.615126
36  0.987633  0.987633
37  1.596078  1.596078
38  3.155941  3.155941
39  0.987633  0.987633
40  1.057143  1.057143
41  1.013293  1.013293
42  1.62096